In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Générer des données simulées
np.random.seed(42)
n_users = 100
n_types = 5

# Créer des utilisateurs avec des types aléatoires
users = pd.DataFrame({
    'user_id': range(n_users),
    'user_type': np.random.randint(0, n_types, n_users)
})

# Générer des notes aléatoires entre utilisateurs
ratings = []
for i in range(n_users):
    for j in range(n_users):
        if i != j:
            rating = np.random.randint(1, 6)  # Notes de 1 à 5
            ratings.append((i, j, rating))

ratings_df = pd.DataFrame(ratings, columns=['user_id', 'rated_user_id', 'rating'])

# Créer une matrice de notes utilisateur-utilisateur
rating_matrix = ratings_df.pivot(index='user_id', columns='rated_user_id', values='rating').fillna(0)

# Calculer la similarité cosinus entre les utilisateurs
user_similarity = cosine_similarity(rating_matrix)

def get_recommendations(user_id, top_n=5):
    user_scores = user_similarity[user_id]
    similar_users = user_scores.argsort()[::-1][1:]  # Exclure l'utilisateur lui-même
    
    user_type = users.loc[user_id, 'user_type']
    
    potential_partners = []
    competitors = []
    
    for similar_user in similar_users:
        if similar_user in users.index:
            if users.loc[similar_user, 'user_type'] != user_type:
                potential_partners.append(similar_user)
            else:
                competitors.append(similar_user)
        
        if len(potential_partners) >= top_n and len(competitors) >= top_n:
            break
    
    potential_partners = users.loc[potential_partners[:top_n]]
    competitors = users.loc[competitors[:top_n]]
    
    return potential_partners, competitors

# Exemple d'utilisation
user_id = 0
potential_partners, competitors = get_recommendations(user_id)

print(f"Partenaires potentiels pour l'utilisateur {user_id}:")
print(potential_partners)
print(f"\nConcurrents potentiels pour l'utilisateur {user_id}:")
print(competitors)

# Ajouter des coordonnées géographiques simulées pour la visualisation sur une carte
def add_geo_coordinates(df):
    df['lat'] = np.random.uniform(48.8, 49.0, len(df))  # Latitude approximative de Paris
    df['lng'] = np.random.uniform(2.3, 2.4, len(df))  # Longitude approximative de Paris
    return df

users = add_geo_coordinates(users)
potential_partners = add_geo_coordinates(potential_partners)
competitors = add_geo_coordinates(competitors)

print("\nDonnées des utilisateurs avec coordonnées géographiques:")
print(users.head())
print("\nPartenaires potentiels avec coordonnées:")
print(potential_partners)
print("\nConcurrents avec coordonnées:")
print(competitors)


Partenaires potentiels pour l'utilisateur 0:
    user_id  user_type
30       30          2
59       59          1
67       67          1
25       25          2
46       46          1

Concurrents potentiels pour l'utilisateur 0:
    user_id  user_type
52       52          3
51       51          3
42       42          3
58       58          3
49       49          3

Données des utilisateurs avec coordonnées géographiques:
   user_id  user_type        lat       lng
0        0          3  48.807718  2.355183
1        1          4  48.975776  2.363177
2        2          2  48.862987  2.325712
3        3          4  48.825796  2.394865
4        4          4  48.893692  2.316976

Partenaires potentiels avec coordonnées:
    user_id  user_type        lat       lng
30       30          2  48.956658  2.337631
59       59          1  48.992557  2.336319
67       67          1  48.825331  2.305227
25       25          2  48.884126  2.368554
46       46          1  48.896188  2.343621

Concurrent